## Objective

Now that I know that my submodels perform a bit better than the model on the entire dataset, I can take advantage of the submodels and find the coefficients that are most associated with response variable.

In [1]:
CATEGORY_GROUPS_IN_QUESTION = [['Pick up Dead Animal', 'Animal Generic Request'],
['Abandoned Vehicles', 'Abandoned Bicycle'],
['Rodent Activity',	'Bed Bugs', 'Mice Infestation - Residential'],
['Sidewalk Repair', 'Sidewalk Repair (Make Safe)'],
['Needle Pickup'],
['Unsatisfactory Living Conditions', 'Poor Conditions of Property', 'Unsanitary Conditions - Establishment', 'Illegal Occupancy', 'Heat - Excessive  Insufficient'],
['Request for Pothole Repair'],
['Graffiti Removal']]

## Objective

Does this have statistically significant coefs, assuming homoskedacity and a linear predictor-response relationship and Normalized residuals and imperfect collinearity?

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from __future__ import division
import pandas as pd
import numpy as np
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
from pylab import rcParams
%matplotlib inline
from tqdm import tqdm

from utilities import remove_one_feature

warnings.filterwarnings("ignore", category=DeprecationWarning)
sns.set_style("whitegrid")
sns.set_context("poster")
rcParams['figure.figsize'] = 20, 5

import os, sys
sys.path.append(os.path.join(os.path.dirname('.'), "../preprocessing"))
from helper_functions import dummify_cols_and_baselines, make_alphas, remove_outliers_by_type, adjusted_r2, transform_school, get_vifs

/home/ubuntu/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
df_orig = pd.read_pickle('../data/data_from_remove_from_dataset.pkl')
df_orig.shape

(516406, 40)

In [5]:
df_orig = transform_school(df_orig)
df_orig.shape

../preprocessing/helper_functions.py:58: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  df.school = df.school.str.extract(r'(\d\d?)').astype(int)


(516406, 40)

## Filterering by `TYPE`

In [6]:
i = 6
print CATEGORY_GROUPS_IN_QUESTION[i]
df_orig = df_orig[df_orig.TYPE.isin(CATEGORY_GROUPS_IN_QUESTION[i])]
df_orig.shape

['Request for Pothole Repair']


(12985, 40)

## Removing outliers

A standard procedure is to remove values further than 3 standard deviations from the mean. Since I have so many low values and some very high values, I anecdotally think that the low values are very likely to be true, and the high values not so much.

So, I will remove values further than 3 SDs from the median, by type.

Ideally, I would take into account the time dimension. I would like to do so given more time.

In [7]:
df_outliers_removed = remove_outliers_by_type(df_orig, y_col='COMPLETION_HOURS_LOG_10')
df_outliers_removed.shape

(12795, 40)

## Remove `TYPE` col

In [8]:
df_outliers_removed.drop('TYPE', axis=1, inplace=True)

## Choosing columns

In [ ]:
['SubmittedPhoto',
 'race_asian',
 'race_hispanic',
 'race_other',
 'poverty_pop_below_poverty_level',
 'earned_income_per_capita',
 'poverty_pop_w_food_stamps',
 'school',
 'bedroom',
 'rent',
 'income_std_dev',
 'Source_Constituent Call']

In [68]:
cols_orig_dataset = ['COMPLETION_HOURS_LOG_10', 'Property_Type', 'Source']
cols_census = [
     'poverty_pop_w_public_assistance',
     'poverty_pop_w_ssi',
     'school_std_dev',
     'housing',
     'bedroom_std_dev',
     'value',
     'value_std_dev',
     'rent_std_dev',
     'income',
]
cols_engineered = ['queue_wk', 'queue_wk_open', 'is_description']

In [69]:
df = df_outliers_removed[cols_orig_dataset + cols_census + cols_engineered]
df.shape

(12795, 15)

## Removing NAs for cols like `school_std_dev`

In [70]:
aa = df.isnull().any().reset_index()
nas = aa[aa[0] == True]['index']
print nas

5      school_std_dev
7     bedroom_std_dev
9       value_std_dev
10       rent_std_dev
Name: index, dtype: object


In [71]:
# this is a bad temporary band-aid
df = df.dropna(subset=nas.tolist())
df.shape

(11775, 15)

## Dummify

In [72]:
cols_to_dummify = [i for i in df.dtypes[df.dtypes == object].index if i != 'TYPE']
cols_to_dummify

['Property_Type', 'Source', 'housing']

In [73]:
df_dummified, baseline_cols = dummify_cols_and_baselines(df, cols_to_dummify, chosen_col_i=2)

Intersection is baseline 0 3
Self Service is baseline 1 3
own is baseline 2 3


In [58]:
df_dummified.shape

(11775, 16)

## Checking for multicollinearity

In [96]:
df_dummified.head(1).T

,905400
COMPLETION_HOURS_LOG_10,0.0124857
SubmittedPhoto,False
poverty_pop_below_poverty_level,0.262473
bedroom,2
queue_wk,12873
queue_wk_open,1
is_description,True


In [16]:
get_vifs(df_dummified.drop(['SubmittedPhoto', 'is_description'], axis=1), 'COMPLETION_HOURS_LOG_10')

poverty_pop_w_food_stamps          5.169946
poverty_pop_below_poverty_level    3.535689
earned_income_per_capita           3.147087
Source_Citizens Connect App        2.743074
income                             2.679695
Source_Constituent Call            2.660618
race_black                         2.390102
poverty_pop_w_ssi                  2.376285
school                             2.174995
housing_std_dev                    2.001001
income_std_dev                     1.966447
race_hispanic                      1.867750
poverty_pop_w_public_assistance    1.770233
school_std_dev                     1.736923
rent_std_dev                       1.736251
value_std_dev                      1.699919
rent                               1.639594
race_asian                         1.538817
value                              1.437490
bedroom                            1.419468
bedroom_std_dev                    1.296181
race_other                         1.134955
queue_wk                        

## Running model

In [17]:
from sklearn.cross_validation import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.cross_validation import ShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import string
from StringIO import StringIO


In [18]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [19]:
X_train, X_test, y_train, y_test = train_test_split(
    df_dummified.drop('COMPLETION_HOURS_LOG_10', axis=1), 
    df_dummified.COMPLETION_HOURS_LOG_10, 
    test_size=0.2, 
    random_state=300
)

## Use LassoCV to find col subsets

In [20]:
pipe = make_pipeline(StandardScaler(), LassoCV())
cv = ShuffleSplit(X_train.shape[0], n_iter=1, test_size=0.2, random_state=300)

In [23]:
params = {'lassocv__alphas': make_alphas(-2, 0)}
model = GridSearchCV(pipe, param_grid=params, n_jobs=-1, cv=cv, verbose=0)
model.fit(X_train, y_train)
pd.DataFrame(model.cv_results_).T.iloc[2:5]

,0,1,2,3,4
mean_test_score,0.0646574,0.0522921,0.00685614,-1.94718e-05,-1.94718e-05
mean_train_score,0.0602605,0.0483346,0.00626461,0,0
param_lassocv__alphas,[0.01],[0.03],[0.1],[0.3],[1.0]


In [24]:
model.best_params_

{'lassocv__alphas': [0.01]}

In [25]:
'{} cols go to zero out of {}'.format(
    len(X_train.columns[model.best_estimator_.steps[-1][-1].coef_ == 0]),
    len(X_train.columns)
)

'12 cols go to zero out of 27'

In [26]:
cols_zero = list(X_train.columns[model.best_estimator_.steps[-1][-1].coef_ == 0])
cols_zero

['SubmittedPhoto',
 'race_asian',
 'race_hispanic',
 'race_other',
 'poverty_pop_below_poverty_level',
 'earned_income_per_capita',
 'poverty_pop_w_food_stamps',
 'school',
 'bedroom',
 'rent',
 'income_std_dev',
 'Source_Constituent Call']

## Use subsetted cols to run lin reg

In [74]:
df_dummified.columns = [col.translate(None, string.punctuation).replace(' ', '') if col != 'COMPLETION_HOURS_LOG_10' else col for col in df_dummified.columns]

In [75]:
X_train, X_test, y_train, y_test = train_test_split(
    df_dummified.drop(['COMPLETION_HOURS_LOG_10'], axis=1), 
    df_dummified.COMPLETION_HOURS_LOG_10, 
    test_size=0.2, 
    random_state=300
)

In [76]:
col_list = ' + '.join(df_dummified.drop(['COMPLETION_HOURS_LOG_10'], axis=1).columns)

est = smf.ols(
    'COMPLETION_HOURS_LOG_10 ~ {}'.format(col_list), 
    pd.concat([X_train, y_train], axis=1)).fit()

In [37]:
est.summary().tables[0]

Dep. Variable:,COMPLETION_HOURS_LOG_10,R-squared:,0.065
Model:,OLS,Adj. R-squared:,0.063
Method:,Least Squares,F-statistic:,34.56
Date:,"Thu, 23 Feb 2017",Prob (F-statistic):,1.85e-122
Time:,18:30:54,Log-Likelihood:,-10442.
No. Observations:,9420,AIC:,2.092e+04
Df Residuals:,9400,BIC:,2.107e+04
Df Model:,19,,
Covariance Type:,nonrobust,,


### Getting adjusted $R^2$ on test set

In [77]:
y_pred = est.predict(X_test)

In [78]:
adjusted_r2(y_test, y_pred, num_features=X_test.shape[1])

0.032435684675251319

In [79]:
mean_squared_error(y_test, y_pred)**0.5

0.74900721906035217

## Interpreting model

Which features are most associated with completion time?

In [82]:
df_results = pd.read_csv(StringIO(est.summary().tables[1].as_csv()), index_col=0).reset_index()
df_results.columns = ['coef_name'] + [i.rstrip().lstrip() for i in df_results.columns][1:]
df_results.coef_name = df_results.coef_name.map(lambda x: x.strip())
df_results = df_results.sort_values('P>|t|')
df_results['pct_diff_for_y'] = (10**df_results.coef - 1) * 100
df_results['pct_diff_for_y_abs'] = pd.np.abs((10**df_results.coef - 1) * 100)
df_results.sort_values('pct_diff_for_y_abs', inplace=True, ascending=False)
df_results.shape

(15, 8)

In [83]:
df_results

,coef_name,coef,std err,t,P>|t|,[95.0% Conf. Int.],pct_diff_for_y,pct_diff_for_y_abs
4,schoolstddev,2.813100e+00,4.810000e-01,5.849,0.000,1.870 3.756,64927.940547,64927.940547
5,bedroomstddev,1.211800e+00,2.040000e-01,5.940,0.000,0.812 1.612,1528.545887,1528.545887
0,Intercept,9.605000e-01,7.000000e-02,13.709,0.000,0.823 1.098,813.061435,813.061435
3,povertypopwssi,2.952000e-01,1.280000e-01,2.307,0.021,0.044 0.546,97.333128,97.333128
8,rentstddev,-1.280500e+00,2.980000e-01,-4.299,0.000,-1.864 -0.697,-94.757964,94.757964
7,valuestddev,-9.462000e-01,2.880000e-01,-3.288,0.001,-1.510 -0.382,-88.681210,88.681210
2,povertypopwpublicassistance,-3.947000e-01,1.900000e-01,-2.079,0.038,-0.767 -0.022,-59.700468,59.700468
1,isdescription[T.True],1.987000e-01,2.200000e-02,8.854,0.000,0.155 0.243,58.015613,58.015613
13,SourceCitizensConnectApp,-1.294000e-01,2.600000e-02,-4.983,0.000,-0.180 -0.078,-25.766489,25.766489
14,SourceConstituentCall,-1.003000e-01,2.600000e-02,-3.885,0.000,-0.151 -0.050,-20.622028,20.622028


In [67]:
scores = []

for col in X_train.columns:
    if col != 'Intercept':
        score = remove_one_feature([col], df_dummified)
        scores.append((col, score))
        
sorted(scores, key=lambda x: x[1])[::-1]        

[('valuestddev', 45),
 ('housingstddev', 45),
 ('povertypopwpublicassistance', 42.837),
 ('value', 42.835),
 ('SourceConstituentCall', 42.719),
 ('income', 42.68),
 ('isdescription', 42.662),
 ('queuewkopen', 40.778999999999996),
 ('PropertyTypeAddress', 40.735),
 ('rentstddev', 40.665),
 ('povertypopwssi', 40.389),
 ('schoolstddev', 39.966),
 ('queuewk', 38.689),
 ('bedroomstddev', 38.669),
 ('SourceCitizensConnectApp', 37.94)]

In [84]:
df_results[df_results['P>|t|'] < 0.1][df_results.coef_name != 'Intercept']

,coef_name,coef,std err,t,P>|t|,[95.0% Conf. Int.],pct_diff_for_y,pct_diff_for_y_abs
4,schoolstddev,2.813100e+00,4.810000e-01,5.849,0.000,1.870 3.756,64927.940547,64927.940547
5,bedroomstddev,1.211800e+00,2.040000e-01,5.940,0.000,0.812 1.612,1528.545887,1528.545887
3,povertypopwssi,2.952000e-01,1.280000e-01,2.307,0.021,0.044 0.546,97.333128,97.333128
8,rentstddev,-1.280500e+00,2.980000e-01,-4.299,0.000,-1.864 -0.697,-94.757964,94.757964
7,valuestddev,-9.462000e-01,2.880000e-01,-3.288,0.001,-1.510 -0.382,-88.681210,88.681210
2,povertypopwpublicassistance,-3.947000e-01,1.900000e-01,-2.079,0.038,-0.767 -0.022,-59.700468,59.700468
1,isdescription[T.True],1.987000e-01,2.200000e-02,8.854,0.000,0.155 0.243,58.015613,58.015613
13,SourceCitizensConnectApp,-1.294000e-01,2.600000e-02,-4.983,0.000,-0.180 -0.078,-25.766489,25.766489
14,SourceConstituentCall,-1.003000e-01,2.600000e-02,-3.885,0.000,-0.151 -0.050,-20.622028,20.622028
12,PropertyTypeAddress,-4.870000e-02,1.600000e-02,-2.978,0.003,-0.081 -0.017,-10.607723,10.607723


## Selected Findings

- **Greater diversity in # of yrs of school, # bedrooms** associated with **worse** completion time, but **greater diversity in rent, house value** associated with **better** completion time.
- ...

This is a weird categ too, for the same reasons as Needle Pickup. Going to skip for now...